# Tutorial 1

more detailed docs: https://marctoussaint.github.io/robotics-course/

In [1]:
import sys
sys.path.append('../build')
print(sys.version)
import numpy as np
import libry as ry

3.8.10 (default, Sep 28 2021, 16:10:42) 
[GCC 9.3.0]
**ry-c++-log** /home/yongxi/Workspace/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback** INFO:/home/yongxi/Workspace/rai-python/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'
** INFO:/home/yongxi/Workspace/rai-python/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/yongxi/Workspace/rai-python/tutorials'
** INFO:/home/yongxi/Workspace/rai-python/rai/rai/Core/graph.cpp:initParameters:1363(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1}



** INFO:/home/yongxi/Workspace/rai-python/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


## Setting up a basic Config

The starting point is to create a `Configuration`.

In [2]:
C = ry.Config()
C.view()

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

You can add things (objects, scene models, robots) to a configuration.

In [3]:
C.clear()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')

NO SELECTION: SELECTION DEPTH = 1 200


Note that the view was updated automatically.

In [4]:
ball = C.addFrame(name="ball")
ball.setShape(ry.ST.sphere, [.1])
ball.setPosition([.8,.8,1.5])
ball.setColor([1,1,0])

One can also add convex meshes (just passing the vertex array), or use sphere-swept convex meshes (ssBox, capsule, sphere, etc)

In [5]:
hand = C.addFrame(name="hand", parent="pr2L")
hand.setShape(ry.ST.ssBox, size=[.2,.2,.1,.02]),
hand.setRelativePosition([0,0,-.1])
hand.setColor([1,1,0])

In this last example, the new object has another frame (pr2L) as *parent*. This means that it is permanently attached to this parent. pos and quat/rot are interpreted relative to the parent.

In [6]:
f = C.frame("hand")
print("position:", f.getPosition())
print("orientation:", f.getQuaternion())

position: [-0.15365539  0.56413119  0.96711955]
orientation: [ 0.33464431 -0.27540941 -0.72687736  0.53274023]


## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [7]:
q = C.getJointState()
print('joint names: ', C.getJointNames() )
print('joint state: ', q)

joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'torso_lift_joint', 'head_pan_joint', 'laser_tilt_mount_joint', 'r_shoulder_pan_joint', 'l_shoulder_pan_joint', 'head_tilt_joint', 'r_shoulder_lift_joint', 'l_shoulder_lift_joint', 'r_upper_arm_roll_joint', 'l_upper_arm_roll_joint', 'r_elbow_flex_joint', 'l_elbow_flex_joint', 'r_forearm_roll_joint', 'l_forearm_roll_joint', 'r_wrist_flex_joint', 'l_wrist_flex_joint', 'r_wrist_roll_joint', 'l_wrist_roll_joint', 'r_gripper_l_finger_jointr_gripper_r_finger_jointr_gripper_l_finger_tip_jointr_gripper_r_finger_tip_joint', 'l_gripper_l_finger_jointl_gripper_r_finger_jointl_gripper_l_finger_tip_jointl_gripper_r_finger_tip_joint', 'r_gripper_joint', 'l_gripper_joint']
joint state:  [ 0.          0.          1.57079633  0.1         0.          0.
 -1.          1.          0.4         0.5         0.5        -1.
  1.         -2.         -2.         -1.5         1.5        -0.5
 -0.5        -0.5 

Let's move the configuration by adding to the joint configuration

In [8]:
q[2] = q[2] + 1.
C.setJointState(q)

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [9]:
X0 = C.getFrameState()
print('frame state: ', X0)

frame state:  [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.95954963]
 [ 0.04207355 -0.02701512  0.890675   ...  0.67850405 -0.19907851
   0.67850405]
 ...
 [ 0.5         2.          0.4        ...  0.          0.
   0.        ]
 [ 0.8         0.8         1.5        ...  0.          0.
   0.        ]
 [-0.55772038  0.17550483  0.96711955 ...  0.10678907 -0.7699332
   0.62796056]]


Let's do a questionable thing: adding .1 to all numbers in the frame matrix!

In [10]:
X = X0 + .1
C.setFrameState(X)

The rows of X have non-normalized quaternions! These are normalized when setting the frame state.

Also, the frame poses are now *inconsistent* to the joint constraints! We can read out the projected joint state, set the joint state, and get a consistent state again:

In [11]:
C.setJointState( C.getJointState() )

Now all *joint* transformations are consistent: just hingeX transformations or alike. However, all the other relative transformations of links and shapes are still messed up from setting their frame pose. Let's bring the configuration back into the state before the harsh *setFrame*

In [12]:
C.setFrameState(X0)

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-select joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q.

The frame state is not affected by such a selection of active joints.

In [13]:
C.selectJointsByTag(["armL","base"])
q = C.getJointState()
print('joint state: ', q)
print('joint names: ', C.getJointNames() )

joint state:  [ 0.          0.          2.57079633  1.00001     0.50001     1.00001
 -2.00001     1.50001    -0.50001     0.50001   ]
joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'l_shoulder_pan_joint', 'l_shoulder_lift_joint', 'l_upper_arm_roll_joint', 'l_elbow_flex_joint', 'l_forearm_roll_joint', 'l_wrist_flex_joint', 'l_wrist_roll_joint']


## Features & Jacobians

A core part of rai defines features over configurations. A feature is a differentiable mapping from a configuration (or set of configurations) to a vector. Starndard features are "position-of-endeffector-X" or "distance/penetration-between-convex-shapes-A-and-B", etc. But there are many, many more features defined in rai, like error of Newton-Euler-equations for an object, total energy of the system, etc. Defining differentiable features is the core of many functionalities in the rai code.

Let's define a basic feature over C: the 3D (world coordinate) position of pr2L (left hand)

In [14]:
F = C.feature(ry.FS.position, ["pr2L"])

We can now evaluate the feature, and also get the Jacobian:

In [15]:
print(F.description(C))

[y,J] = F.eval(C)
print('hand position:', y)
print('Jacobian:', J)
print('Jacobian shape:', J.shape)

F_Position/0-pr2L
hand position: [-0.55020139  0.07798996  0.94627935]
Jacobian: [[ 1.          0.         -0.22809985 -0.26435692  0.11135503  0.04779693
  -0.48205508 -0.33495284 -0.02817069 -0.07290386]
 [ 0.          1.         -0.7156038  -0.49224536  0.16348124  0.120796
   0.00421212 -0.02651866 -0.0452186  -0.00259795]
 [ 0.          0.         -0.07364733 -0.00287152 -0.09559552 -0.31855228
  -0.16469275 -0.06076456  0.20142175 -0.01414692]]
Jacobian shape: (3, 10)


We can linearly transform features by setting 'scale' and 'target':

In [16]:
F.setScale([10.]) #note: needs to be an array!
F.setTarget([0., 0., 1.])
F.eval(C)

(array([-5.50201394,  0.77989957, -0.53720654]),
 array([[10.        ,  0.        , -2.28099853, -2.64356925,  1.11355032,
          0.47796926, -4.82055078, -3.34952843, -0.28170688, -0.72903865],
        [ 0.        , 10.        , -7.15603803, -4.92245359,  1.63481236,
          1.20796002,  0.04212124, -0.26518657, -0.45218602, -0.02597949],
        [ 0.        ,  0.        , -0.7364733 , -0.02871517, -0.95595517,
         -3.18552282, -1.64692747, -0.60764563,  2.0142175 , -0.14146922]]))

Setting scale and target actually transforms the feature to become
$$
  \phi(x) \gets \texttt{scale} \cdot (\phi(x) - \texttt{target})
$$
The $\cdot$ is flexibly interpreted: scale can be an arbitrary matrix, or a vector of same size as $\phi$ (element-wise re-scaling), or a single scalar (as above). E.g., if we only care about the height of the robot hand, we can do this:

In [17]:
F.setScale([[0,0,10]])      #pick z-coordinate only, and scale by 10
F.setTarget([0., 0., 1.])   #note that the target needs to be in UNSCALED space!
F.eval(C)

(array([-0.53720654]),
 array([[ 0.        ,  0.        , -0.7364733 , -0.02871517, -0.95595517,
         -3.18552282, -1.64692747, -0.60764563,  2.0142175 , -0.14146922]]))

A negative-distance (penetration) feature example

In [18]:
F2 = C.feature(ry.FS.distance, ["hand", "ball"]) #distance actually means neg-distance
print(F2.description(C))

F_PairCollision/0-hand-ball


In [19]:
F2.eval(C)

(array([-1.35583742]),
 array([[ 0.85716403,  0.40723476, -0.58442865, -0.49809181,  0.1340833 ,
         -0.0032576 , -0.54509652, -0.36514016,  0.00672904, -0.06356971]]))

When a feature is of higher *order*, by default it computes the difference, acceleration, etc, w.r.t. multiple configurations. We need to create a compound configurations ("tuple") of several configurations to evaluate this:

In [20]:
C2 = ry.Config()
C2.addConfigurationCopy(C,tau=1.)  #this replicates the whole structure, but with semantics 'tau'-translated in time
C2.addConfigurationCopy(C,tau=1.)  #this replicates the whole structure
V2 = C2.view()

In [21]:
F.setScale([])  #reset to default (no scaling)
F.setTarget([]) #reset to default (zero target)
F.setOrder(1)
F.eval(C2)[0]

array([0., 0., 0.])

This should be zero, as the difference between the first and second copy is zero. To see a difference, let's move the 2nd configuration:

In [ ]:
q = C.getJointState()
q = q - .1
C2.setJointStateSlice(q,1)
y = F.eval(C2)[0]
print('hand difference (y(C2) - y(K)) =', y)

An acceleration example:

In [ ]:
C2.addConfigurationCopy(C);

In [ ]:
C2.setJointStateSlice(q + .3, 2);

In [ ]:
F.setOrder(2)
(y,J) = F.eval(C2)
print('hand acceleration:', y)
print('shape of Jacobian:', J.shape)
print(F.description(C2))

Note that the Jacobian is now w.r.t. all three configurations! It is of size 3x(3xdim(q)). Let's retrieve the Jacobian w.r.t. C3 only:

In [ ]:
J = J.reshape((3,3,q.size))
print('shape of Jacobian:', J.shape)
J[:,1,:]

In [ ]:
C=0

## Editing Configurations

In [ ]:
C = ry.Config()
C.view()

In [ ]:
import subprocess
subprocess.Popen(['gedit', 'tmp.g'])

In [ ]:
## uncomment this! (Automatic testing for this interactive method does not work...)
#C.edit('tmp.g')

In [ ]:
C=0